In [43]:
import pandas as pd

In [44]:
bnb = pd.read_csv("bnb_results.csv")
lr = pd.read_csv("lr_results.csv")
cnn = pd.read_csv("cnn_results.csv")
bert = pd.read_csv("bert_results.csv")

In [45]:
bnb.head()

,Unnamed: 0,Comment,Result,Comment_Adj,No_Stop,Stemmed,Result_Bin,Predicted_Result,Predicted_Probability
0,0,I feel like we became too dependent on our rec...,Loss,i feel like we became too dependent on our rec...,feel like became dependent recent late-game he...,feel like becam depend recent late-gam heroic ...,0,1,0.578302
1,1,I like it even more when I don't think we're g...,Win,i like it even more when i don't think we're g...,like even n't think 're gon na win win anyways,like even n't think 're gon na win win anyway,1,1,0.999777
2,2,This game confirmed everything I already knew ...,Loss,this game confirmed everything i already knew ...,game confirmed everything already knew bobby ’...,game confirm everyth alreadi knew bobbi ’ play...,0,0,0.300150
3,3,I’m in shock as a Blazers fan. I know for a fa...,Loss,i’m in shock as a blazers fan i know for a fac...,’ shock blazers fan know fact lillard play way...,’ shock blazer fan know fact lillard play way ...,0,1,0.999483
4,4,Can we please change the banner to our current...,Win,can we please change the banner to our current...,please change banner current bucks roster ’ wa...,pleas chang banner current buck roster ’ want ...,1,1,0.999648


In [46]:
lr.head()

,Unnamed: 0,Comment,Result,Comment_Adj,No_Stop,Stemmed,Result_Bin,Predicted_Result,Predicted_Probability
0,0,I feel like we became too dependent on our rec...,Loss,i feel like we became too dependent on our rec...,feel like became dependent recent late-game he...,feel like becam depend recent late-gam heroic ...,0,0,0.225801
1,1,I like it even more when I don't think we're g...,Win,i like it even more when i don't think we're g...,like even n't think 're gon na win win anyways,like even n't think 're gon na win win anyway,1,1,0.911172
2,2,This game confirmed everything I already knew ...,Loss,this game confirmed everything i already knew ...,game confirmed everything already knew bobby ’...,game confirm everyth alreadi knew bobbi ’ play...,0,0,0.305607
3,3,I’m in shock as a Blazers fan. I know for a fa...,Loss,i’m in shock as a blazers fan i know for a fac...,’ shock blazers fan know fact lillard play way...,’ shock blazer fan know fact lillard play way ...,0,1,0.705575
4,4,Can we please change the banner to our current...,Win,can we please change the banner to our current...,please change banner current bucks roster ’ wa...,pleas chang banner current buck roster ’ want ...,1,1,0.958090


In [47]:
cnn.head()

,Unnamed: 0,Comment,Result,Comment_Adj,No_Stop,Stemmed,Result_Bin,Actual_Label,Predicted_Label
0,0,I feel like we became too dependent on our rec...,Loss,i feel like we became too dependent on our rec...,feel like became dependent recent late-game he...,feel like becam depend recent late-gam heroic ...,0,0,1
1,1,I like it even more when I don't think we're g...,Win,i like it even more when i don't think we're g...,like even n't think 're gon na win win anyways,like even n't think 're gon na win win anyway,1,1,1
2,2,This game confirmed everything I already knew ...,Loss,this game confirmed everything i already knew ...,game confirmed everything already knew bobby ’...,game confirm everyth alreadi knew bobbi ’ play...,0,0,0
3,3,I’m in shock as a Blazers fan. I know for a fa...,Loss,i’m in shock as a blazers fan i know for a fac...,’ shock blazers fan know fact lillard play way...,’ shock blazer fan know fact lillard play way ...,0,0,0
4,4,Can we please change the banner to our current...,Win,can we please change the banner to our current...,please change banner current bucks roster ’ wa...,pleas chang banner current buck roster ’ want ...,1,1,1


In [67]:
bnb_tp = bnb[(bnb["Result_Bin"] == 1) & (bnb["Predicted_Result"] == 1)]
lr_fn = lr[(lr["Result_Bin"] == 1) & (lr["Predicted_Result"] == 0)]
cnn_fn = cnn[(cnn["Result_Bin"] == 1) & (cnn["Predicted_Label"] == 0)]
bert_fn = bert[(bert["Result_Bin"] == 1) & (bert["Predicted"] == 0)]

In [68]:
first_200_df = bnb_tp.sort_values("Predicted_Probability", ascending = True)
first_200 = first_200_df['Comment'].head(200)

In [69]:
comments_lr = first_200.isin(lr_fn['Comment'])
comments_cnn = first_200.isin(cnn_fn['Comment'])

comments_both = first_200[comments_lr & comments_cnn]

In [70]:
comments_both

19      Big response against one of the hottest teams ...
1132    This is my last remaining team-specific subred...
2114    I point out when he plays well, and I’m gonna ...
2180    I think what's happening is coaching is trying...
1489    Problem with being the best is every single te...
962     Dame’s game is really unbelievable… He’s the b...
1655    He was questionable with non-covid illness. I ...
2068    I’m going to keep saying this, Doc is just Coa...
567     B2B and all those excuses aside, this goes to ...
647     So we're going to beat the good teams & lose t...
1629    Fr his playmaking has been nothing short of am...
2234    Yeah, we were 19-6 at this point last season a...
446     If dame played like this in Portland, we would...
2016    I will take the L, but you still think this te...
2236    Correction: Giannis would’ve bullied AG into g...
1044    It's because of the strength of schedule i gue...
544     Teams already sag off him. They celebrate if h...
1243    Both g

In [71]:
with pd.option_context('display.max_colwidth', None):
    print(comments_both)

19                                                                                                                                                                                         Big response against one of the hottest teams in the Association right now. Cavs will cause a lot of problems especially if fully healthy. We’ve got a lot of uncertainties with how we will organize as a team but we have the answer to the biggest question: do we have the talent to hang with any team?  Good W!
1132                                                                                                                                                                                        This is my last remaining team-specific subreddit, the others became too toxic so I unsubbed. This one is heading there too, sadly. I think the Dame trade brought in a bunch of sad Packers fans (there are many) and spoiled what used to be my favorite sports subreddit. I'm still hopeful it'll bounce back ...
2114  

In [48]:
bnb_fp = bnb[(bnb["Result_Bin"] == 0) & (bnb["Predicted_Result"] == 1)]
lr_tn = lr[(lr["Result_Bin"] == 0) & (lr["Predicted_Result"] == 0)]
cnn_tn = cnn[(cnn["Result_Bin"] == 0) & (cnn["Predicted_Label"] == 0)]

In [49]:
lr_tn.head()

,Unnamed: 0,Comment,Result,Comment_Adj,No_Stop,Stemmed,Result_Bin,Predicted_Result,Predicted_Probability
0,0,I feel like we became too dependent on our rec...,Loss,i feel like we became too dependent on our rec...,feel like became dependent recent late-game he...,feel like becam depend recent late-gam heroic ...,0,0,0.225801
2,2,This game confirmed everything I already knew ...,Loss,this game confirmed everything i already knew ...,game confirmed everything already knew bobby ’...,game confirm everyth alreadi knew bobbi ’ play...,0,0,0.305607
5,5,Good fight on the road against the defending c...,Loss,good fight on the road against the defending c...,good fight road defending champs bad first gam...,good fight road defend champ bad first game doc,0,0,0.061192
6,6,Doc is overrated. Major L recruiting him,Loss,doc is overrated major l recruiting him,doc overrated major l recruiting,doc overr major l recruit,0,0,0.343441
8,8,Yep! We got a lot of bucks basketball left woo...,Loss,yep we got a lot of bucks basketball left woot...,yep got lot bucks basketball left woot hope do...,yep got lot buck basketbal left woot hope dont...,0,0,0.436663


In [50]:
first_200_df = bnb_fp.sort_values("Predicted_Probability", ascending = False)
first_200 = first_200_df['Comment'].head(200)

In [51]:
comments_lr = first_200.isin(lr_tn['Comment'])
comments_cnn = first_200.isin(cnn_tn['Comment'])

comments_both = first_200[comments_lr & comments_cnn]

In [52]:
with pd.option_context('display.max_colwidth', None):
    print(comments_both)

2223    Livingston had some decent moments on both ends tonight. Hopefully can see some improvement with more minutes
1373                                                                                 They used to be bad… really bad…
1876                                                                                    Rather him than 3-13 dame tbh
1213                                              Having watched this game: Yes. The Bucks are really fucking stupid.
580                                         If you can't suggest players in the proposed trade, then this is useless.
Name: Comment, dtype: object


In [54]:
target_comment = "Livingston"

# Find the comment in the dataframe
found_comment = lr_tn[lr_tn['Comment'].str.contains(target_comment)]

# Print the found comment
print(found_comment)

      Unnamed: 0                                            Comment Result  \
892          892  If the scheme that you designed is so bad that...   Loss   
921          921  I can't much blame Livingston for his offense ...   Loss   
929          929  I think there's probs some hesitation to shoot...   Loss   
1809        1809  MarJon was already out of the rotation - and I...   Loss   
2223        2223  Livingston had some decent moments on both end...   Loss   

                                            Comment_Adj  \
892   if the scheme that you designed is so bad that...   
921   i can't much blame livingston for his offense ...   
929   i think there's probs some hesitation to shoot...   
1809  marjon was already out of the rotation  and i ...   
2223  livingston had some decent moments on both end...   

                                                No_Stop  \
892   scheme designed bad chris livingston best play...   
921   ca n't much blame livingston offense game cons...   

In [55]:
target_comment = "Livingston"

# Find the comment in the dataframe
found_comment = lr[lr['Comment'].str.contains(target_comment)]

# Print the found comment
print(found_comment)

      Unnamed: 0                                            Comment Result  \
892          892  If the scheme that you designed is so bad that...   Loss   
921          921  I can't much blame Livingston for his offense ...   Loss   
929          929  I think there's probs some hesitation to shoot...   Loss   
1075        1075  Wiggins doesn’t think he’s a superstar and is ...    Win   
1209        1209  My dream of Chris Livingston Jr becoming a rot...    Win   
1235        1235  The Blazers are good at their developing their...   Loss   
1809        1809  MarJon was already out of the rotation - and I...   Loss   
2223        2223  Livingston had some decent moments on both end...   Loss   

                                            Comment_Adj  \
892   if the scheme that you designed is so bad that...   
921   i can't much blame livingston for his offense ...   
929   i think there's probs some hesitation to shoot...   
1075  wiggins doesn’t think he’s a superstar and is ...   
12

In [41]:
target_comment = "Livingston had some decent moments on both ends tonight. Hopefully can see some improvement with more minutes"

# Find the comment in the dataframe
found_comment = bnb_fp[bnb['Comment'].str.contains(target_comment)]

# Print the found comment
print(found_comment)

      Unnamed: 0                                            Comment Result  \
2223        2223  Livingston had some decent moments on both end...   Loss   

                                            Comment_Adj  \
2223  livingston had some decent moments on both end...   

                                                No_Stop  \
2223  livingston decent moments ends tonight hopeful...   

                                                Stemmed  Result_Bin  \
2223  livingston decent moment end tonight hope see ...           0   

      Predicted_Result  Predicted_Probability  
2223                 1               0.999957  


/var/folders/hs/br_4rpdj68nc3sfdpgv0xgn80000gn/T/ipykernel_24579/3657591789.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  found_comment = bnb_fp[bnb['Comment'].str.contains(target_comment)]


In [53]:
target_comment = "Livingston had some decent moments on both ends tonight. Hopefully can see some improvement with more minutes"

# Find the comment in the dataframe
found_comment = lr_tn[lr_tn['Comment'].str.contains(target_comment)]

# Print the found comment
print(found_comment)

      Unnamed: 0                                            Comment Result  \
2223        2223  Livingston had some decent moments on both end...   Loss   

                                            Comment_Adj  \
2223  livingston had some decent moments on both end...   

                                                No_Stop  \
2223  livingston decent moments ends tonight hopeful...   

                                                Stemmed  Result_Bin  \
2223  livingston decent moment end tonight hope see ...           0   

      Predicted_Result  Predicted_Probability  
2223                 0               0.382528  


In [2]:
import pickle
from ...functions.functions_utils import process_text, basic_process, cnn_process

ImportError: attempted relative import with no known parent package

In [ ]:
def return_coef_bnb(text):

    model = bnb
    text = process_text(input_field)
    text = bnb_vectorizer.transform([text])

    vocabulary = vectorizer.get_feature_names_out()
    coefficients = model.coef_[0]

    # Map coefficients to words
    word_coefficient_map = {word: coef for word, coef in zip(vocabulary, coefficients)}

    # Print coefficients for words in the input text
    for word in input_text.split():
        if word in word_coefficient_map:
            print(f"Word: {word}, Coefficient: {word_coefficient_map[word]}")
        else:
            print(f"Word: {word}, Coefficient: 0")  